## Preprocessing Steps
**Spike removal / filtering methods**
<br>
    -- Reduction of spike events by special design of the instrument (Zhao, 2003)
<br>
    -- Automatic Spike Removal Algorithm for Raman Spectra: wavelet transform (spike removal raman filter from matlab)
<br>
    -- Missing point polynomial filter (I have the code)
<br>
    -- Robust smoothing filter
<br>
    -- Moving window filter 
<br>
**Remove background Autofluorescence noise**
<br>
--IModPoly (Chad A Lieber and Anita Mahadevan-Jansen. Automated method for subtraction offluorescence from biological raman spectra.Applied spectroscopy, 57(11):1363–1367,2003) (https://github.com/michaelstchen/modPolyFit)(Faster technique)
 <br>
--Zhiming Zhang (An intelligent background-correction algorithm for highly fluorescent samples in raman spectroscopy: https://onlinelibrary.wiley.com/doi/abs/10.1002/jrs.2500)(https://github.com/zmzhang/baselineWavelet)
<br>
--Vancouver Raman Algorithm (Jianhua Zhao: http://journals.sagepub.com/doi/abs/10.1366/000370207782597003) 
<br>
--EMD (Empirical  Mode Decomposition) (https://github.com/laszukdawid/PyEMD)
<br>
**Smoothing (Denoising)**
<br>
-- Savisky-Golay filtering (Scipi package):  https://github.com/scipy/scipy/blob/master/scipy/signal/_savitzky_golay.py
<br>
-- Moving Average/median
<br>
--CARS (Coherent Anti-Stokes Raman spectroscopy) 
<br>
**Normalize**
<br>
--Min/Max method (I have the code).
<br>
--Vector based 
<br>
**Spectral and intensity re-calibration**

**Normal**
<br>
Individual patients with 5 sample points in blood is 471
<br>
Individual patients with 3 sample points in blood is 228

**Disease 1:**

Individual patients with 5 sample points in blood is 153.
<br>
Individual patients with 3 sample points in blood is 20.


In [1]:
'''
Class dealing with the Raman data
'''
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
from imblearn.over_sampling import SMOTE
import random
import os
import pickle
import pandas as pd
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from convertwdf import *
from wdfReader import * 
from scipy import sparse
from scipy.sparse.linalg import spsolve
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, MaxPooling1D, Bidirectional,LSTM
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPool1D, Flatten , Embedding, GlobalMaxPool1D
from keras.models import Model
from keras.optimizers import SGD, Adam, rmsprop
#%matplotlib inline 
#https://github.com/MacDumi/Deconvolution
#python3 Deconvolution_test.py /home/titli/Documents/Deconvolution-master/0151.txt 
#https://www.pnas.org/content/114/31/8247

/home/titli/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def normalize(data):
    _min = np.min(data)
    _max = np.max(data)
    return (data - _min) / (_max - _min)
def getspikes(fileID):
    
    x_data= fileID.get_xdata()
    spectra= fileID.get_spectra()
    return (x_data, spectra)

In [3]:
patient_array_1 = [] #patients in disease1
patient_array_0 = [] #patients in disease0
spectra_array0 = [] #spectrum in disease0
spectra_array1 = [] #spectrum in disease1

In [4]:
#Disease 1
rootdir = '/home/titli/Documents/disease1'
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #print (os.path.join(subdir, file))
        txt = os.path.join(subdir, file)
        x = txt.split("/")
        if( x[5] == '1_0-5-1' and x[8] == '980'):
            if (str(x[7]) not in patient_array_1):
                patient_array_1.append(x[7])
                wdfIle = wdfReader(os.path.join(subdir, file))
                X, spectra = getspikes(wdfIle) # plotting the spectrum
                #if len(spectra)<1015:
                #    spectra[len(spectra):len(spectra)+(1015-len(spectra))]=10000
                spectra = normalize(spectra)
                spectra_array1.append(spectra)
spectra_array_1= pd.DataFrame(spectra_array1)
labels_1 = pd.DataFrame({'labels': np.ones((len(spectra_array1),), dtype=int)})

### Normal patients

In [5]:
rootdir = '/home/titli/Documents/normal'
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #print (os.path.join(subdir, file))
        txt = os.path.join(subdir, file)
        x = txt.split("/")
        if( x[5] == '1_0-5-1' and x[8] == '980'):
            if (str(x[7]) not in patient_array_0):
                patient_array_0.append(x[7])
                wdfIle = wdfReader(os.path.join(subdir, file))
                X, spectra = getspikes(wdfIle) # plotting the spectrum
                #if len(spectra)<1015:
                #    spectra[len(spectra):len(spectra)+(1015-len(spectra))]=10000
                spectra = normalize(spectra)
                spectra_array0.append(spectra)
spectra_array_0= pd.DataFrame(spectra_array0)
labels_0 = pd.DataFrame({'labels': np.zeros((len(spectra_array0),), dtype=int)})

In [6]:
spectra_array_0.head(10)

,0,1,2,3,4,5,6,7,8,9,...,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014
0,0.011308,0.006305,0.008163,0.010361,0.000854,0.010665,0.006273,0.005631,0.010779,0.010136,...,0.410484,0.409580,0.404766,0.403591,0.406505,0.406420,0.415236,0.417964,0.416877,0.408439
1,0.018628,0.000000,0.008605,0.006683,0.010225,0.011337,0.004359,0.016999,0.015077,0.016996,...,0.702732,0.710878,0.698623,0.721818,0.683514,0.697643,0.698664,0.716016,0.717518,0.708675
2,0.009522,0.013726,0.009025,0.010740,0.008650,0.007976,0.002351,0.001546,0.003438,0.000512,...,0.135977,0.136347,0.133178,0.133158,0.132856,0.134464,0.131475,0.137076,0.136278,0.140992
3,0.001365,0.006520,0.004609,0.014125,0.011093,0.000000,0.014213,0.017899,0.018674,0.006022,...,0.743194,0.731239,0.735599,0.732072,0.732309,0.750455,0.741198,0.742148,0.743993,0.741006
4,0.013971,0.010878,0.010359,0.010013,0.010440,0.007218,0.006186,0.002451,0.006826,0.003864,...,0.173007,0.180340,0.175647,0.175660,0.176670,0.181112,0.177315,0.181686,0.179331,0.181814
5,0.004867,0.000776,0.003694,0.003351,0.000757,0.000000,0.001671,0.002809,0.000513,0.004965,...,0.158370,0.153636,0.163266,0.158154,0.160056,0.163236,0.159264,0.165093,0.167228,0.162265
6,0.009930,0.007167,0.006735,0.010690,0.007388,0.005603,0.000950,0.003983,0.005012,0.006907,...,0.159082,0.158425,0.161016,0.166725,0.167105,0.167356,0.170549,0.174000,0.174724,0.179297
7,0.010439,0.013195,0.006978,0.007971,0.010553,0.003387,0.007133,0.001136,0.006470,0.001639,...,0.412738,0.409432,0.415960,0.418079,0.408418,0.420618,0.418415,0.416212,0.417904,0.428737
8,0.017110,0.004732,0.004036,0.000000,0.006852,0.007640,0.009665,0.005383,0.005801,0.006712,...,0.431166,0.434050,0.435448,0.431997,0.438242,0.430738,0.425313,0.429479,0.423365,0.427529
9,0.015331,0.012443,0.008660,0.007925,0.016812,0.014313,0.019828,0.012842,0.024764,0.000000,...,0.662620,0.666214,0.659030,0.656851,0.652109,0.662239,0.651602,0.668007,0.650072,0.663011


In [7]:
total_df = pd.concat([spectra_array_0,spectra_array_1], axis = 0)
total_df.fillna(1e-5)
#too many zeros replace with a min value of 1e-4 to avoid nan in loss function
total_df = total_df.apply(lambda x: [y if y <= 1e-5 else 1e-4 for y in x])
labels_df = pd.concat([labels_0,labels_1], axis = 0)
indices=list(range(0,len(total_df)))
random.shuffle(indices)
X = total_df.values[indices]
y = labels_df.values[indices]
#
#len(total_df)

In [8]:
# Test- Train Dataset: Making a balanced dataset 50 disease1 and 50 normal
split_val= int(len(X)*0.8)
X_train=X[:split_val]
X_test=X[split_val:,]
y_train =y[:split_val]
y_test =y[split_val:]
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
# Convert labels to categorical one-hot encoding
y_train_labels = to_categorical(y_train, num_classes=2)
y_test_labels = to_categorical(y_test, num_classes=2)

In [9]:
def kraub_method():
    inp =  Input(shape=(1015, 1))
    x = Conv1D(32, kernel_size = 7, strides= 1,padding='valid', activation='relu')(inp)
    x = Conv1D(16, kernel_size = 5, strides= 1, padding='valid', activation='relu')(x)
    x = Flatten()(x)
    x = Dropout(0.01)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.01)(x)
    x = Dense(256, activation='relu')(x)
    preds = Dense(2, activation='softmax')(x)
    model = Model(inp, preds)
    model.compile(loss= 'categorical_crossentropy',
              optimizer= 'Adam',
              metrics=['acc'])
    return model
    

In [10]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_model_step2.hdf5".format('boat_detector')
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=10) 
callbacks_list = [checkpoint, early, reduceLROnPlat]

/home/titli/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [11]:
model = kraub_method()
history = model.fit(X_train, y_train_labels, batch_size= 10, epochs=65, validation_data=(X_test, y_test_labels),callbacks=callbacks_list)

Train on 420 samples, validate on 105 samples
Epoch 1/65
420/420 [==============================] - 4s 10ms/step - loss: 0.6075 - acc: 0.7167 - val_loss: 0.6700 - val_acc: 0.7048

Epoch 00001: val_loss improved from inf to 0.66998, saving model to boat_detector_model_step2.hdf5
Epoch 2/65
420/420 [==============================] - 3s 8ms/step - loss: 0.6023 - acc: 0.7310 - val_loss: 0.6073 - val_acc: 0.7048

Epoch 00002: val_loss improved from 0.66998 to 0.60731, saving model to boat_detector_model_step2.hdf5
Epoch 3/65
420/420 [==============================] - 3s 8ms/step - loss: 0.5887 - acc: 0.7310 - val_loss: 0.6131 - val_acc: 0.7048

Epoch 00003: val_loss did not improve from 0.60731
Epoch 4/65
420/420 [==============================] - 3s 8ms/step - loss: 0.5968 - acc: 0.7310 - val_loss: 0.6218 - val_acc: 0.7048

Epoch 00004: val_loss did not improve from 0.60731
Epoch 5/65
420/420 [==============================] - 3s 8ms/step - loss: 0.5850 - acc: 0.7310 - val_loss: 0.6425 - v

In [12]:
model_json = model.to_json()
with open("model_step1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_step1.h5")
print("Saved model to disk")

Saved model to disk


In [13]:
patient_array_1 = [] #patients in disease1
patient_array_0 = [] #patients in disease0
spectra_array0 = [] #spectrum in disease0
spectra_array1 = [] #spectrum in disease1
#Disease 1
rootdir = '/home/titli/Documents/test/disease1'
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #print (os.path.join(subdir, file))
        txt = os.path.join(subdir, file)
        x = txt.split("/")
        if( x[6] == '1_0-5-1' and x[9] == '980'):
            if (str(x[8]) not in patient_array_1):
                patient_array_1.append(x[8])
                wdfIle = wdfReader(os.path.join(subdir, file))
                X, spectra = getspikes(wdfIle) # plotting the spectrum
                #if len(spectra)<1015:
                #    spectra[len(spectra):len(spectra)+(1015-len(spectra))]=10000
                spectra = normalize(spectra)
                spectra_array1.append(spectra)
spectra_array_1= pd.DataFrame(spectra_array1)
labels_test_1 = pd.DataFrame({'labels': np.ones((len(spectra_array1),), dtype=int)})
rootdir = '/home/titli/Documents/test/normal'
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #print (os.path.join(subdir, file))
        txt = os.path.join(subdir, file)
        x = txt.split("/")
        if( x[6] == '1_0-5-1' and x[9] == '980'):
            if (str(x[8]) not in patient_array_0):
                patient_array_0.append(x[8])
                wdfIle = wdfReader(os.path.join(subdir, file))
                X, spectra = getspikes(wdfIle) # plotting the spectrum
                #if len(spectra)<1015:
                #    spectra[len(spectra):len(spectra)+(1015-len(spectra))]=10000
                spectra = normalize(spectra)
                spectra_array0.append(spectra)
spectra_array_0= pd.DataFrame(spectra_array0)
labels_test_0 = pd.DataFrame({'labels': np.zeros((len(spectra_array0),), dtype=int)})

In [14]:
total_df_test = pd.concat([spectra_array_0,spectra_array_1], axis = 0)
X_test = total_df_test.values
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
labels_df_test = pd.concat([labels_test_0,labels_test_1], axis = 0)
y_test = labels_df_test.values
y_test = to_categorical(y_test, num_classes=2)
model1_test_y = model.predict(X_test, batch_size=10, verbose=1)

110/110 [==============================] - 0s 1ms/step


In [17]:
model1_test_y[model1_test_y > 0.5] = 1
model1_test_y[model1_test_y <= 0.5] = 0

/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in less_equal
  


In [18]:
def F1_score(pred_test_y, actuals):

    predictions =[]
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0
    
    for i in range (len(pred_test_y)):
        if ((pred_test_y[i,0]==1) & (actuals[i,0]==1)):
            true_pos = true_pos+1
        elif((pred_test_y[i,0]==0) & (actuals[i,0]==0)):
            true_neg = true_neg+1
        elif((pred_test_y[i,0]==1) & (actuals[i,0]==0)):
            false_pos = false_pos +1
        elif((pred_test_y[i,0]==0) & (actuals[i,0]==1)):
            false_neg = false_neg+1
    prec=true_pos/(true_pos+false_pos)
    recall = true_pos/(true_pos+false_neg)
    accur=(true_pos+true_neg)/(true_pos+false_pos+ true_neg+ false_neg)
    #F1=2*(prec*recall/(prec+recall))
    #FPR = false_pos/(false_pos+true_neg)
    return (true_pos, false_pos, true_neg, false_neg, accur)

In [19]:
print((F1_score(model1_test_y, y_test)))

(90, 0, 0, 0, 1.0)
